In [1]:
import sqlite3
import pandas as pd
import math
from scipy import stats
import seaborn as sns
import numpy as np
import matplotlib as plt

In [2]:
connection = sqlite3.connect('Northwind_small.sqlite')
cursor = connection.cursor()

In [3]:
def sql_select_to_df(SQL_COMMAND, cursor=cursor):
    """returns output of SQL command in dataframe format"""
    results = cursor.execute(SQL_COMMAND).fetchall()
    df = pd.DataFrame(results)
    df.columns = [i[0] for i in cursor.description]
    return df

In [4]:
 product_orderDetails_df = sql_select_to_df("""SELECT *
                                FROM [Product] AS P
                                JOIN [OrderDetail] AS O
                                ON P.ID = O.ProductID
                        """)

product_orderDetails_df

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,11,Queso Cabrales,5,4,1 kg pkg.,21.00,22,30,30,0,10248/11,10248,11,14.00,12,0.00
1,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,26,0,0,1,10248/42,10248,42,9.80,10,0.00
2,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80,14,0,0,0,10248/72,10248,72,34.80,5,0.00
3,14,Tofu,6,7,40 - 100 g pkgs.,23.25,35,0,0,0,10249/14,10249,14,18.60,9,0.00
4,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10249/51,10249,51,42.40,40,0.00
5,41,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65,85,0,10,0,10250/41,10250,41,7.70,10,0.00
6,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10250/51,10250,51,42.40,35,0.15
7,65,Louisiana Fiery Hot Pepper Sauce,2,2,32 - 8 oz bottles,21.05,76,0,0,0,10250/65,10250,65,16.80,15,0.15
8,22,Gustaf's Knäckebröd,9,5,24 - 500 g pkgs.,21.00,104,0,25,0,10251/22,10251,22,16.80,6,0.05
9,57,Ravioli Angelo,26,5,24 - 250 g pkgs.,19.50,36,0,20,0,10251/57,10251,57,15.60,15,0.05


In [5]:
#Extract category IDs
category_IDs = product_orderDetails_df['CategoryId'].values
category_IDs = list(category_IDs)

#Extract revenues
prices = product_orderDetails_df.iloc[:,13].values
quantities = product_orderDetails_df['Quantity'].values

revenues = prices * quantities
revenues = list(revenues)
print({'categoryIDs':category_IDs ,'revenues':revenues})

{'categoryIDs': [4, 5, 4, 7, 7, 8, 7, 2, 5, 5, 2, 3, 4, 4, 4, 1, 3, 1, 6, 7, 1, 3, 8, 4, 6, 2, 3, 1, 2, 1, 2, 4, 3, 8, 8, 5, 3, 1, 3, 1, 2, 7, 5, 3, 1, 8, 7, 1, 8, 6, 1, 4, 8, 4, 1, 6, 4, 4, 4, 8, 1, 4, 3, 4, 4, 8, 4, 4, 8, 1, 4, 4, 1, 4, 8, 8, 7, 3, 2, 4, 2, 8, 6, 1, 6, 1, 3, 1, 1, 8, 5, 2, 3, 4, 4, 3, 2, 4, 1, 1, 8, 6, 1, 3, 3, 1, 8, 6, 3, 2, 5, 2, 6, 3, 2, 8, 2, 7, 3, 8, 1, 2, 1, 1, 6, 1, 4, 1, 5, 4, 3, 4, 1, 4, 1, 8, 4, 3, 3, 1, 2, 3, 8, 5, 6, 7, 1, 8, 2, 3, 3, 4, 4, 8, 6, 1, 8, 6, 6, 3, 3, 4, 1, 2, 2, 5, 1, 4, 3, 3, 5, 4, 7, 1, 6, 1, 8, 4, 8, 3, 1, 1, 8, 3, 1, 8, 1, 6, 7, 1, 4, 1, 5, 2, 3, 1, 3, 1, 8, 4, 2, 2, 8, 7, 4, 4, 2, 5, 1, 1, 4, 8, 8, 4, 2, 3, 3, 8, 1, 5, 3, 4, 6, 8, 5, 3, 7, 3, 4, 5, 3, 1, 4, 4, 7, 2, 5, 3, 8, 8, 4, 6, 8, 2, 6, 3, 8, 8, 1, 4, 4, 1, 4, 8, 6, 5, 3, 1, 2, 2, 2, 3, 5, 6, 5, 3, 1, 8, 1, 1, 5, 6, 3, 4, 1, 8, 2, 2, 1, 6, 4, 1, 1, 6, 1, 5, 4, 6, 4, 8, 3, 4, 1, 1, 8, 3, 4, 4, 7, 6, 1, 3, 6, 1, 4, 3, 7, 6, 4, 1, 1, 4, 4, 4, 2, 2, 1, 6, 2, 2, 3, 7, 5, 5, 6, 5, 1, 5,

In [18]:
pd.options.display.max_rows = 9999
product_orderDetails_df

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,11,Queso Cabrales,5,4,1 kg pkg.,21.00,22,30,30,0,10248/11,10248,11,14.00,12,0.00
1,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,26,0,0,1,10248/42,10248,42,9.80,10,0.00
2,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80,14,0,0,0,10248/72,10248,72,34.80,5,0.00
3,14,Tofu,6,7,40 - 100 g pkgs.,23.25,35,0,0,0,10249/14,10249,14,18.60,9,0.00
4,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10249/51,10249,51,42.40,40,0.00
5,41,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65,85,0,10,0,10250/41,10250,41,7.70,10,0.00
6,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10250/51,10250,51,42.40,35,0.15
7,65,Louisiana Fiery Hot Pepper Sauce,2,2,32 - 8 oz bottles,21.05,76,0,0,0,10250/65,10250,65,16.80,15,0.15
8,22,Gustaf's Knäckebröd,9,5,24 - 500 g pkgs.,21.00,104,0,25,0,10251/22,10251,22,16.80,6,0.05
9,57,Ravioli Angelo,26,5,24 - 250 g pkgs.,19.50,36,0,20,0,10251/57,10251,57,15.60,15,0.05


In [7]:
unique_productIDs = sorted(product_orderDetails_df['CategoryId'].unique())
print(unique_productIDs)

[1, 2, 3, 4, 5, 6, 7, 8]


In [8]:
def get_revenues_by_category(categoryNum, revenues_list, IDs_list):
    """Loops through revenues list and pushes relevant figure to list to be returned."""
    master_list = []
    iterator_count = 0
    
    for i in IDs_list:
        if i == categoryNum:
            master_list.append(revenues_list[iterator_count])
    
        iterator_count+=1
    
    return master_list
            

id1_revenues = get_revenues_by_category(1,revenues,category_IDs)
id2_revenues = get_revenues_by_category(2,revenues,category_IDs)
id3_revenues = get_revenues_by_category(3,revenues,category_IDs)
id4_revenues = get_revenues_by_category(4,revenues,category_IDs)
id5_revenues = get_revenues_by_category(5,revenues,category_IDs)
id6_revenues = get_revenues_by_category(6,revenues,category_IDs)
id7_revenues = get_revenues_by_category(7,revenues,category_IDs)
id8_revenues = get_revenues_by_category(8,revenues,category_IDs)

In [9]:
def get_experimental_and_control_population(id_num, arr_of_groups):
    """Returns: 
        experimental dataset: category ID input (id_num argument),
        control dataset: rest of category ID groups i.e. selected id removed. (arr_of_groups argument)    
    """
    
    id_num_index = id_num - 1

    def get_control_population(generic):
        master_list = []
        for g in generic:
            master_list.append(g)
    
        return master_list

    control = get_control_population(arr_of_groups)

    return {'experimental':control.pop(id_num_index), 'control':control}

#Example: If you want control and experimental groups for revenues from categoryID 1
populations = get_experimental_and_control_population(1,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
populations

{'experimental': [604.8000000000001,
  54.0,
  304.0,
  86.4,
  760.0,
  252.0,
  288.0,
  100.8,
  532.0,
  240.0,
  216.0,
  919.9999999999999,
  475.2,
  43.2,
  43.2,
  186.0,
  21.6,
  57.6,
  56.0,
  648.0,
  1440.0,
  224.0,
  36.0,
  37.2,
  259.2,
  552.0,
  37.2,
  864.0,
  608.0,
  240.0,
  441.59999999999997,
  432.0,
  60.0,
  736.0,
  883.1999999999999,
  62.0,
  156.79999999999998,
  360.0,
  288.0,
  86.4,
  432.0,
  144.0,
  57.6,
  1008.0,
  310.0,
  380.0,
  4216.0,
  106.39999999999999,
  1472.0,
  364.79999999999995,
  216.0,
  720.0,
  37.2,
  216.0,
  4216.0,
  36.0,
  10540.0,
  172.8,
  90.0,
  36.0,
  112.0,
  2108.0,
  777.6,
  74.4,
  172.8,
  403.2,
  216.0,
  8432.0,
  288.0,
  360.0,
  54.0,
  112.0,
  54.0,
  360.0,
  576.0,
  380.0,
  432.0,
  504.0,
  504.0,
  144.0,
  345.6,
  201.6,
  10540.0,
  912.0,
  96.0,
  864.0,
  10329.2,
  288.0,
  259.2,
  152.0,
  148.8,
  224.0,
  684.0,
  115.2,
  86.4,
  72.0,
  300.0,
  288.0,
  736.0,
  24.8,
  110.39

In [10]:
category1_data = get_experimental_and_control_population(1,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category1_data_experimental = category1_data['experimental']
category1_data_control = category1_data['control']

category2_data = get_experimental_and_control_population(2,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category2_data_experimental = category2_data['experimental']
category2_data_control = category2_data['control']

category3_data = get_experimental_and_control_population(3,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category3_data_experimental = category3_data['experimental']
category3_data_control = category3_data['control']

category4_data = get_experimental_and_control_population(4,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category4_data_experimental = category4_data['experimental']
category4_data_control = category4_data['control']

category5_data = get_experimental_and_control_population(5,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category5_data_experimental = category5_data['experimental']
category5_data_control = category5_data['control']

category6_data = get_experimental_and_control_population(6,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category6_data_experimental = category6_data['experimental']
category6_data_control = category6_data['control']

category7_data = get_experimental_and_control_population(7,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category7_data_experimental = category7_data['experimental']
category7_data_control = category7_data['control']

category8_data = get_experimental_and_control_population(8,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category8_data_experimental = category8_data['experimental']
category8_data_control = category8_data['control']

In [11]:
def one_sample_ttest(sample_data, population_data, alpha):
    
    #Population mean
    mu_calc_figures = []
        
    for p in population_data:
        for elem in p:
            mu_calc_figures.append(elem)
        
    mu = sum(mu_calc_figures)/len(mu_calc_figures)
    
    x_bar = sum(sample_data)/len(sample_data)
    
    #Sigma
    sample_data_arr = np.asarray(sample_data)
    sigma = np.std(sample_data_arr)
    
    #Degrees of Freedom
    df = len(sample_data) -1
    
    #Sample size
    n = len(sample_data)
    
    #Critical t value
    t_crit_positive = stats.t.ppf(1 - alpha, df=df)
    t_crit_negative = t_crit_positive * -1
    
    #T statistic
    t_stat_numerator = x_bar - mu
    t_stat_denominator = sigma/math.sqrt(n)
    t_stat = t_stat_numerator/t_stat_denominator
    
    if int(t_stat*100) in range(int(t_crit_negative*100), int(t_crit_positive*100)):
        return 'fail to reject the null hypothesis'
    elif int(t_stat*100) < int(t_crit_negative*100):
        return 'reject the null hypothesis. Sample mean is significantly less than population mean'
    elif int(t_stat*100) > int(t_crit_positive*100):
        return 'reject the null hypothesis. Sample mean is significantly greater than population mean'
    print(t_stat)

In [12]:
category1_test_outcome = one_sample_ttest(category1_data_experimental, category1_data_control, 0.05)
category2_test_outcome = one_sample_ttest(category2_data_experimental, category2_data_control, 0.05)
category3_test_outcome = one_sample_ttest(category3_data_experimental, category3_data_control, 0.05)
category4_test_outcome = one_sample_ttest(category4_data_experimental, category4_data_control, 0.05)
category5_test_outcome = one_sample_ttest(category5_data_experimental, category5_data_control, 0.05)
category6_test_outcome = one_sample_ttest(category6_data_experimental, category6_data_control, 0.05)
category7_test_outcome = one_sample_ttest(category7_data_experimental, category7_data_control, 0.05)
category8_test_outcome = one_sample_ttest(category8_data_experimental, category8_data_control, 0.05)

In [13]:
print(f'Test outcome of category 1: {category1_test_outcome}')
print(f'Test outcome of category 2: {category2_test_outcome}')
print(f'Test outcome of category 3: {category3_test_outcome}')
print(f'Test outcome of category 4: {category4_test_outcome}')
print(f'Test outcome of category 5: {category5_test_outcome}')
print(f'Test outcome of category 6: {category4_test_outcome}')
print(f'Test outcome of category 7: {category7_test_outcome}')
print(f'Test outcome of category 8: {category8_test_outcome}')

Test outcome of category 1: fail to reject the null hypothesis
Test outcome of category 2: reject the null hypothesis. Sample mean is significantly less than population mean
Test outcome of category 3: reject the null hypothesis. Sample mean is significantly less than population mean
Test outcome of category 4: reject the null hypothesis. Sample mean is significantly greater than population mean
Test outcome of category 5: reject the null hypothesis. Sample mean is significantly less than population mean
Test outcome of category 6: reject the null hypothesis. Sample mean is significantly greater than population mean
Test outcome of category 7: reject the null hypothesis. Sample mean is significantly greater than population mean
Test outcome of category 8: reject the null hypothesis. Sample mean is significantly less than population mean


In [14]:
categories = sql_select_to_df("""SELECT *
                    FROM [Product] AS P
                    JOIN [Category] AS C
                    ON P.CategoryID = C.ID
                        """)

categories = categories.iloc[:,2:12]
categories.head()

,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,Id,CategoryName
0,1,1,10 boxes x 20 bags,18.00,39,0,10,0,1,Beverages
1,1,1,24 - 12 oz bottles,19.00,17,40,25,0,1,Beverages
2,1,2,12 - 550 ml bottles,10.00,13,70,25,0,2,Condiments
3,2,2,48 - 6 oz jars,22.00,53,0,0,0,2,Condiments
4,2,2,36 boxes,21.35,0,0,0,1,2,Condiments


In [15]:
#categories = categories.drop(['SupplierId','QuantityPerUnit','UnitPrice','UnitsInStock','UnitsOnOrder','ReorderLevel','Discontinued','Id'], axis=1)
categories

def extract_categoryNames_from_categoryId(unique_ids, df):
    id_count = 1
    
    categoryNames = []
    IDs = []
    
    for i in unique_ids:
        df_relevant = df.loc[df['CategoryId'] == i]
        categoryName = df_relevant['CategoryName'].values[0]
        
        categoryNames.append(categoryName)
        IDs.append(i)
        
    parent_list = []
    j=0
    while j < len(IDs):
        child_dict = {'CategoryName': categoryNames[j], 'ID': IDs[j]}
        print(child_dict)
        parent_list.append(child_dict)
        j+=1
    
    return parent_list
    
categories_final = extract_categoryNames_from_categoryId(unique_productIDs, categories)

{'CategoryName': 'Beverages', 'ID': 1}
{'CategoryName': 'Condiments', 'ID': 2}
{'CategoryName': 'Confections', 'ID': 3}
{'CategoryName': 'Dairy Products', 'ID': 4}
{'CategoryName': 'Grains/Cereals', 'ID': 5}
{'CategoryName': 'Meat/Poultry', 'ID': 6}
{'CategoryName': 'Produce', 'ID': 7}
{'CategoryName': 'Seafood', 'ID': 8}


In [16]:
high_performing_categories = categories_final
high_performing_categories

[{'CategoryName': 'Beverages', 'ID': 1},
 {'CategoryName': 'Condiments', 'ID': 2},
 {'CategoryName': 'Confections', 'ID': 3},
 {'CategoryName': 'Dairy Products', 'ID': 4},
 {'CategoryName': 'Grains/Cereals', 'ID': 5},
 {'CategoryName': 'Meat/Poultry', 'ID': 6},
 {'CategoryName': 'Produce', 'ID': 7},
 {'CategoryName': 'Seafood', 'ID': 8}]

In [17]:
width = .35 # width of a bar

m1_t = pd.DataFrame({
 'Mean' : [90,40,30,30,30,25,25,20,15,10],
 'Condiments' : [60,70,65,70,70,60,50,45,45,45],
 'Confections' : [140,160,170,180,190,200,210,220,230,240],
 'Produce' : [210,100,100,70,70,75,70,60,65,60]})

m1_t[['abnormal','fix','normal','bad_rate']].plot(kind='bar', width = width)
m1_t['bad_rate'].plot(secondary_y=True)

ax = plt.gca()
plt.xlim([-width, len(m1_t['normal'])-width])
ax.set_xticklabels(('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'))

plt.show()

KeyError: "['abnormal' 'fix' 'normal' 'bad_rate'] not in index"